In [ ]:
from collections import OrderedDict
from time import time
import re
import json
from natsort import natsorted
from glob import glob
from pathlib import Path
import os

import math
from random import random, shuffle, choice

# import pydot
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from networkx.drawing.nx_pydot import read_dot

import numpy as np
from scipy.spatial import Delaunay

from matplotlib import collections  as mc
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.style.use('seaborn-colorblind')
from IPython.display import clear_output

nodePattern = re.compile('^(\d+) ')
edgePattern = re.compile('^(\d+) -- (\d+)')
attrsPattern = re.compile('\[(.+)\]')

def isNode(line):
    return '--' not in line


def processAttrs(line):
    attrs = attrsPattern.findall(line)
    if len(attrs) > 0:
        attrs = attrs[0].split(',')
    attrs = [a.split('=') for a in attrs]
    attrs = [[a[0].strip(), a[1].replace('"','').strip()] for a in attrs]
    for a in attrs:
        try:
            a[1] = int(a[1])
        except ValueError:
            try:
                a[1] = float(a[1])
            except:
                pass
    return attrs


def processEdge(line):
    finding = edgePattern.findall(line)[0]
    source, target = finding[:2]
    source, target = int(source), int(target)
    attrs = processAttrs(line)
    return dict(attrs, source=source, target=target)


def processNode(line):
    nodeId = int(line.split(' ')[0])
    nodeAttrs = processAttrs(line)
    return dict(nodeAttrs, id=nodeId)


def draw(g, pos, edges=True, labels=False, figsize=[8,8], s=2, lw=0.5):
    xy = np.array(list([pos[k] for k in g.nodes]))
    fig = plt.figure(figsize=figsize)
    ax = fig.subplots()
    
    ## nodes
    ax.scatter(xy[:,0], xy[:,1], s=s, zorder=3)
    
    ## edges
    if edges:
        lines = [[pos[i], pos[j]] for (i,j) in g.edges]
        lc = mc.LineCollection(lines, colors='grey', linewidths=lw)
        ax.add_collection(lc)
    ax.autoscale()
    ax.margins(0.1)
    plt.axis('equal')

    if labels:
        for i in g.nodes:
            plt.text(pos[i][0], pos[i][1], g.nodes[i]['label'])
    return ax
#     plt.figure(figsize=figsize)
#     nx.draw(
#         g, 
#         pos=pos,
#         node_size=10,
#         width=0.5,
#     )




def subtree_sizes(tree, root):
    tree = nx.bfs_tree(tree, source=root)
    s = [len(nx.bfs_tree(tree, i).nodes) for i in tree.neighbors(root)]
    total = sum(s)
    return np.array(s)


def normalize(node):
    for prop in node:
        if prop == 'pos':
            pos = node[prop].replace('"', '')
            pos = pos.split(',')
            pos = [float(pos[0]), float(pos[1])]
            node[prop] = pos
        else:
            if type(node[prop]) == str:
                node[prop] = node[prop].replace('"', '')
                
            try:
                node[prop] = int(node[prop])
            except ValueError:
                try:
                    node[prop] = float(node[prop])
                except ValueError:
                    pass
            except Exception as err:
                print(err)
                print(node, prop)


## Generate a Graph

In [ ]:
def edges2graph(lines, i2k=None, label2i=None):
    pattern = re.compile('"(.+)" -- "(.+)"')
    nodes = set()
    edges = set()
    for i,line in enumerate(lines):
        if len(line.strip()) > 0:
            edge = re.findall(pattern, line)[0]
            source, target = edge
            nodes.update([source, target])
            edges.add( (source, target) )
    
    if label2i is None:
        label2i = {k:i for i,k in enumerate(nodes)}
        i2k = list(range(len(nodes)))
    g = nx.Graph()
    
    nodes = [dict(id=label2i[k], label=k) for i,k in enumerate(nodes)]
    ids = [n['id'] for n in nodes]
    g.add_nodes_from( zip(ids, nodes) )
    
    edges = [(i2k[label2i[e[0]]],i2k[label2i[e[1]]]) for e in edges]
    g.add_edges_from(edges)
    return g, i2k, label2i


In [ ]:
# for fn in fns:
#     with open(fn) as f:
#         g, i2k, label2i = edges2graph(f.readlines())
#         print(fn, len(g))

In [ ]:
# # fns = natsorted(glob('./data/txt/topics_faryad_500/*.txt'))
# # fns = natsorted(glob('./data/txt/TopicsLayersData-0/*.txt'))
# ## Reyan's level-weight setting
# # nodeCount2levelweight = {
# #     50: (1, 550),
# #     100: (1, 500),
# #     200: (2, 500),
# #     300: (2, 450),
# #     400:(2, 450),
# #     500: (2, 450),
# #     600: (3, 450), 
# #     700: (3, 450),
# #     800: (3, 450),
# #     1200: (3, 400),
# #     1600: (4, 400),
# #     2000: (4, 350),
# #     2500: (5, 350),
# #     3000: (5, 300),
# #     3500: (6, 300),
# #     4000: (6, 250),
# #     4500: (7, 250),
# #     5000: (8, 200)
# # }

# # nodeCount2levelweight = {
# #     50: (1, 550),
# #     100: (2, 500),
# #     200: (3, 500), 
# #     500: (6, 450), 300: (4, 450), 400:(5, 450),
# #     800: (9, 450), 600: (7, 450), 700: (8, 450),
# #     1200: (10, 400),
# #     1600: (11, 400),
# #     2000: (12, 350),
# #     2500: (13, 350),
# #     3000: (14, 300),
# #     3500: (15, 300),
# #     4000: (16, 250),
# #     4500: (17, 250),
# #     5000: (18, 200)
# # }

# nodeCounts = sorted(list(nodeCount2levelweight.keys()))
# levels = [nodeCount2levelweight[nc][0] for nc in nodeCounts]
# weights = [nodeCount2levelweight[nc][1] for nc in nodeCounts]

# list(zip(nodeCounts, levels, weights))

## Choose a graph

In [ ]:

## ## fns = natsorted(glob('./data/txt/topics_refined/*.txt'))[1:]
## ## fns = natsorted(glob('./data/txt/lastfm_refined/*.txt'))
## ## fns = natsorted(glob('./data/txt/topics_steiner/*.txt'))
## ## fns = natsorted(glob('./data/txt/lastfm_steiner/*.txt'))


fns = natsorted(glob('./data/txt/lastfm/*.txt'))
# fns = natsorted(glob('./data/txt/topics_faryad_8level/*.txt'))

# fns = natsorted(glob('./data/txt/tol_graphs/*.txt'))[:8]
# fns = natsorted(glob('./data/txt/tol_graphs/*.txt'))[:8]
# fns = [
#     f'./data/txt/tol_graphs/{fn}' for fn in 
#     [
#         'Graph_0.txt', 
#         'Graph_1.txt', 
#         'Graph_2_600.txt', 
#         'Graph_2.txt', 
#         'Graph_3.txt', 
#         'Graph_4_1600.txt', 
#         'Graph_4_2000.txt',
#         'Graph_4.txt',
#     ]
# ]
#fns = natsorted(glob('./data/txt/math_genealogy/*.txt'))[:4]
# fns = [
#     f'./data/txt/math_genealogy/{fn}' for fn in 
#     [
#       'Graph_0.txt', 
#       'Graph_1.txt', 
#       'Graph_2_600.txt', 
#       'Graph_2.txt', 
#       'Graph_3_1200.txt', 
#       'Graph_3_1600.txt', 
#       'Graph_3_2000.txt', 
#       'Graph_3.txt',
#     ]
# ]

levels = list(range(1, len(fns)+1))
maxLevel = max(*levels)

## linear increment
baseWeight = 200
if 'topics_steiner' in fns[0]:
    increment = 50
elif 'tol_graphs' in fns[0]:
    increment = 50
else:
    increment = 50
weights = [baseWeight+(maxLevel-l)*increment for l in levels]
weights = [w/200 for w in weights]

## exponential increment
# baseWeight = 1
# maxWeight = 1
# incrementFactor = maxWeight**(1/(maxLevel-1))
# weights = [baseWeight*incrementFactor**(maxLevel-l) for l in levels]

list(zip(fns,levels,weights))

In [ ]:
# fn_largest = natsorted(glob('./data/txt/topics_large/*.txt'))[-1]
# print(fn_largest)
# with open(fn_largest) as f:
#     print(len(f.readlines())+1)

In [ ]:
# with open(fns[-1]) as f:
    
#     nodeCount = len(g)
#     level = levels[-1]
#     weight = weights[-1]
#     print(fns[-1], level, nodeCount, weight)
#     for n in g.nodes:
#         g.nodes[n]['level'] = level
#         g.nodes[n]['nodeCount'] = nodeCount
#         g.nodes[n]['weight'] = weight
        
#     for e in g.edges:
#         g.edges[e]['level'] = level
#         g.edges[e]['weight'] = weight

# for fn, level, weight in list(fn_level_weight)[:-1][::-1]:
for i, (fn, level, weight) in list(enumerate(zip(fns, levels, weights)))[::-1]:
    
    with open(fn) as f:
        if level == maxLevel:
            subgraph, i2k, label2i = edges2graph(f.readlines())
            g = subgraph
        else:
            subgraph,_,_ = edges2graph(f.readlines(), i2k, label2i)
        nodeCount = len(subgraph)
        print(fn, level, nodeCount, weight)
    
        for n in subgraph.nodes:
            g.nodes[n]['level'] = level
            g.nodes[n]['nodeCount'] = nodeCount
            g.nodes[n]['weight'] = weight

        for e in subgraph.edges:
            g.edges[e]['level'] = level
            g.edges[e]['weight'] = weight


print('all_pairs_shortest_path...')
apsp = nx.all_pairs_dijkstra_path_length(g, weight='weight')
d = np.zeros([len(g.nodes),len(g.nodes)])
for dk in tqdm(apsp):
    source = dk[0]
    target_dist = dk[1]
    d[source,:] = [target_dist[i] for i in range(len(g.nodes))]
    
    
print('k-hop all_pairs_shortest_path...')
apsp = nx.all_pairs_dijkstra_path_length(g, weight=1)
hops = np.zeros([len(g.nodes),len(g.nodes)])
for dk in tqdm(apsp):
    source = dk[0]
    target_dist = dk[1]
    hops[source,:] = [target_dist[i] for i in range(len(g.nodes))]

fn = fns[-1]

## Initial Layout

In [ ]:
def fan(nodes, 
        origin=[0,0], radii=[], 
        phaseCenter=0, phaseRange=np.pi, 
        weights=[1,1], 
        mode='random'):
    pos = {}
    phases = {}
    ranges = {}
    n = len(nodes)
    cos, sin = np.cos, np.sin
    
    weightTotal = sum(weights)
    weights = [w/weightTotal for w in weights]
    
    nr = sorted(zip(nodes, weights, radii), key=lambda x:x[1])
    
    
    if mode == 'center':
        ## centralize heavy sub trees
        nr2 = []
        for i in list(range(len(nr)))[::-1]:
            if i%2 == 0:
                nr2.append(nr[i])
            else:
                nr2.insert(0, nr[i])
    elif mode == 'polar':
        ## polarize heavy sub trees
        nr2 = []
        for i in range(len(nr)):
            if i%2 == 0:
                nr2.append(nr[i])
            else:
                nr2.insert(0, nr[i])
    elif mode == 'random':
        shuffle(nr)
        nr2 = nr
    elif mode == 'interleave':
        a = nr[::2]
        b = nr[1::2][::-1]
        if len(a)==len(b):
            nr2 = sum(zip(a,b), tuple())
        else:
            nr2 = sum(zip(a,b+[-1,]), tuple())[:-1]
            
    elif mode == 'ordered':
        nr2 = nr
        
    nodes, weights, radii = zip(*nr2)
    
    weightCumSum = [sum(weights[:i]) for i in range(len(weights)+1)]
    for i in range(n):
        angle_offset = (weightCumSum[i]+weightCumSum[i+1])/2 * phaseRange
        angle_i = phaseCenter - phaseRange/2 + angle_offset
        ri = radii[i]
        pos[nodes[i]] = [origin[0] + ri*cos(angle_i), origin[1] + ri*sin(angle_i)]
        phases[nodes[i]] = angle_i
        ranges[nodes[i]] = weights[i] * phaseRange * 0.9
    return pos, phases, ranges


def radial_layout(g, root=None, mode='center', origin=[0,0], phase0=0, range0=np.pi*2):
    g0 = g
    g = nx.bfs_tree(g, source=root)
    pos = {}
    phases = {}
    ranges = {}
    depth_from_root = nx.shortest_path_length(g, root)
    if root is None:
        root = next(iter(g.nodes))
    pos[root] = origin
    phases[root] = phase0
    ranges[root] = range0
    roots = [root, ]
    depth = 1
    while len(pos) < len(g.nodes):
        newRoots = []
        for root in roots:
            if mode=='ordered':
                neighbors = [n for n in g0.nodes[root]['neighbor_order'] if n not in pos]
            else:
                neighbors = [n for n in g.neighbors(root) if n not in pos]
            if len(neighbors) > 0:
                edge_lengths = [g0.edges[(root, n)]['weight'] for n in neighbors]
                subTreeSizes = [len(nx.bfs_tree(g, i).nodes) for i in neighbors]
                degrees = [g.degree[i] for i in neighbors]
                depths = [depth_from_root[i] for i in neighbors]
#                 weights = [(x*z/y**2) for x, y, z in zip(degrees, depths, subTreeSizes)]
                weights = [z for x, y, z in zip(degrees, depths, subTreeSizes)]
#                 neighborSizes = [len(list(g0.neighbors(i))) for i in neighbors]
                newRoots += neighbors
                newPos, newPhases, newRanges = fan(
                    neighbors, 
                    mode=mode,
#                     origin=pos[root], radii=edge_lengths, #mode: Reyan
                    origin=origin, radii=[depth for e in edge_lengths],#mode: mw
                    phaseCenter=phases[root], 
                    phaseRange=ranges[root], 
                    weights=weights,
                )
                pos.update(newPos)
                phases.update(newPhases)
                ranges.update(newRanges)
        roots = newRoots
        depth+=1
    return pos


def rotate(pos0, theta=0):
    cos = math.cos(theta)
    sin = math.sin(theta)
    pos = {}
    for k in pos0:
        p = pos0[k]
        pos[k] = (p[0]*cos-p[1]*sin, p[0]*sin+p[1]*cos)
    return pos



def neighbor_order(nodeId, parentId, neighbors, pos):
    v = np.array([pos[i] for i in neighbors]) - np.array([pos[nodeId]])
    a = np.angle(v[:,0] + 1j * v[:,1])
    order = [neighbors[o] for o in np.argsort(a)]
    if parentId is not None:
        order = np.roll(order, -order.index(parentId))
    return order



In [ ]:
# # root = list(g.nodes)[np.argmin(d.max(axis=1))] ##center node
# root = nx.center(g)[0]
# root

In [ ]:
# %%time

init_layout = 'radial'
# init_layout = 'sfdp-ordered-radial'
# init_layout = 'sfdp'


# # pos0 = nx.layout.planar_layout(g, scale=40)
# # pos0 = graphviz_layout(g, prog="dot", root=list(g.nodes)[0])
# # pos0 = graphviz_layout(g, prog='twopi')

t0 = time()
if init_layout == 'sfdp':
    g1 = nx.Graph(g)
    for i in g1.nodes:
        g1.nodes[i]['label'] = ''
    pos0 = graphviz_layout(g1, prog='sfdp')
    
    
elif init_layout == 'radial':
    root = list(g.nodes)[np.argmin(d.max(axis=1))] ##large-depth node
#     root = choice(list(g.nodes)) ##random node
    
#     root = nx.center(g)[0]
#     root = 1949 ##tree-of-life 3000-nodes
#     root = 1143
    pos0 = radial_layout(g, root, mode='center')
#     pos0 = rotate(pos0, -math.pi*0.5)


elif init_layout == 'sfdp-ordered-radial':
    
    
    ##remove label to avoid parsing bug of sfdp
    g1 = nx.Graph(g)
    for i in g1.nodes:
        g1.nodes[i]['label'] = '' 
        
    pos_sfdp = graphviz_layout(g1, prog='sfdp')
    draw(g, pos_sfdp, s=1, lw=1, labels=False, figsize=[8,8])
#     center = np.array(list(pos_sfdp.values())).mean(0)
#     root = min([p for p in pos_sfdp.items()], key=lambda x:np.linalg.norm(x[1]-center))[0]
    root = list(g.nodes)[np.argmin(d.max(axis=1))] ##large-depth node
    root = 1949 ##tree-of-life 3000-nodes
#     root = nx.center(g)[0]
    bfs = nx.bfs_tree(g, root)
    for nodeId in g:
        try:
            parentId = next(bfs.predecessors(nodeId))
        except StopIteration:
            parentId = None
        neighbors = list(g.neighbors(nodeId))
        g.nodes[nodeId]['neighbor_order'] = neighbor_order(nodeId, parentId, neighbors, pos_sfdp)
    
    pos0 = radial_layout(g, root, mode='ordered')
    


dt = time() - t0
print(f'{dt} sec')


# pos0 = rotate(pos0, math.pi/180* 80)
draw(g, pos0, s=1, lw=1, labels=False, figsize=[24,24])
plt.show()


In [ ]:
# k2i = {v:k for k,v in enumerate(i2k)}
# k2i[root]

In [ ]:
# center = np.array(list(pos0.values())).mean(0)
# root = min([p for p in pos0.items()], key=lambda x:np.linalg.norm(x[1]-center))[0]

## New Ordering of Nodes

In [ ]:
node_order = list(g.nodes)
bfs = nx.bfs_tree(g, root)

### list of node ids


# if ('math-genealogy' in fn 
#     or 'topics-800' in fn
#     or 'covid' in fn
#    ):
    
# print('no re-ordering')
## no re-ordering
# node_order = list(g.nodes)

## order preserving re-sort according to level
# id2level = {i: g.nodes[i]['level'] for i in g.nodes}
# nodeorder_level_pairs = [(i,id2level[i]) for i in node_order]
# node_order = [n[0] for n in sorted(nodeorder_level_pairs, key=lambda x:x[1])]

##bfs ordering
node_order = list(bfs) 



# else:
#     #max degree node
#     print('max degree node re-ordering')
#     degree = list(g.degree)
#     degree = list(zip(range(len(degree)), degree))
#     max_degree_node = max(degree, key=lambda x:x[1][1])
#     start = max_degree_node[1][0]
#     bfs = nx.bfs_tree(g, start)
#     node_order = list(bfs) 
    
# #     # random node, bfs
# #     start = next(iter(g.nodes.keys()))
# #     bfs = nx.bfs_tree(g, start)
# #     node_order = list(bfs) 

# #     #dfs
# #     node_order = list(nx.dfs_preorder_nodes(g, start))





## load dot as networkx graph

In [ ]:
# pos = pos0.copy()
# ideal_edge_length = {
#     e: g.edges[e]['weight'] 
#     for e in g.edges
# }
# actual_edge_length = {
#     (p0,p1): np.linalg.norm(np.array(pos[p1])-np.array(pos[p0]))
#     for (p0,p1) in g.edges
# }
# num = np.sum([ideal_edge_length[k]*actual_edge_length[k] for k in g.edges])
# den = np.sum([actual_edge_length[k]**2 for k in g.edges])
# s = num / den
# print(f's = {s}')

# pos = {k:np.array(v)*s for k,v in pos.items()}

In [ ]:
pos = pos0.copy()
s = 1
pos = {k:list(np.array(v)*s) for k,v in pos.items()}

## to JSON

In [ ]:
suffix = 'linear'

dir_out, name = str(Path(fn).parent), Path(fn).stem
dir_out = dir_out.replace('dot', 'json').replace('txt', 'json')
dir_out += f'_{suffix}'
fn_out = dir_out + '/'+ name
fn_out += f'-{int(time())}'

if not Path(dir_out).exists():
    os.makedirs(Path(dir_out))
else:
    print(Path(dir_out), 'exists')
fn_out

In [ ]:
for n in g.nodes:
    if 'neighbor_order' in g.nodes[n]:
        del g.nodes[n]['neighbor_order']
    else:
        break

In [ ]:
###graph to list
nodes = {k: g.nodes[k] for k in g.nodes}
edges = [[e[0], e[1], g.edges[e]] for e in g.edges]

nodes = [{
    'id': node_order[i],
    'index': i,
    'x': float(pos[node_order[i]][0]),
    'y': float(pos[node_order[i]][1]),
    'neighbors': list(nx.neighbors(g, node_order[i])),
    'perplexity': len(list(nx.neighbors(g, node_order[i]))),
    **nodes[node_order[i]]
} for i in range(len(nodes))]

edges = [{
    'source': e[0],
    'target': e[1],
    **e[2]
} for e in edges]



##store the position & perplexity
for i,node in enumerate(nodes):
    try: 
        parent = next(bfs.predecessors(node['id']))
    except StopIteration:
        parent = None
    node['parent'] = parent

hopThresh = 6
virtual_edges = []
for i in tqdm(range(len(nodes))):
    for j in range(i+1, len(nodes)):
        if d[i,j] == 0:
            print(f'[warning] d[{i},{j}] = 0')
        else:
            if hops[i,j] < hopThresh or random() < 1:
                dij = d[i,j]
                e = {
                    'source': i2k[i],
                    'target': i2k[j],
                    'weight': dij,
                    'hops': hops[i,j]
                }
                virtual_edges.append(e)
            else:
                continue
            
            


    
## option 1: nodes, edges, and virtualEdges with properties
# with open(fn_out, 'w') as f:
#     json.dump(dict(
#         edges = edges, 
#         virtual_edges = virtual_edges, 
#         nodes=nodes
#     ), f, indent=2)
# print('done')


## option 2: properties as arrays
res = {}
for k in nodes[0]:
    print(k)
    res[f'node_{k}'] = [n[k] for n in nodes]
for k in edges[0]:
    res[f'edge_{k}'] = [e[k] for e in edges]

print(fn_out)
with open(fn_out+'-min.json', 'w') as f:
    json.dump(res, f, indent=2)
    
for k in virtual_edges[0]:
    res[f'virtual_edge_{k}'] = [ve[k] for ve in virtual_edges]

print(f'writing {fn_out}.json...')
with open(fn_out+'.json', 'w') as f:
    json.dump(res, f, indent=2)

print('done!')




In [ ]:
##debug

for k,v in res.items():
    print(k, type(res[k][0]))

In [ ]:
# with open('data/json/TopicsLayersData-0/Graph_5000-nodes-3.json') as f:
#     j3 = json.load(f)
# with open('data/json/TopicsLayersData-0/Graph_5000.json') as f:
#     j = json.load(f)   
    

In [ ]:
# nodeIds = [n['id'] for n in nodes]
# nodeLabels = [n['label'] for n in nodes]
# nodeLevels = [n['level'] for n in nodes]
# nodeXs = [n['x'] for n in nodes]
# nodeYs = [n['y'] for n in nodes]
# nodePerplexities = [n['perplexity'] for n in nodes]
# nodeParents = [n['parent'] for n in nodes]
# nodeNeighbors = [n['neighbors'] for n in nodes]
# nodeNodeCount = [n['nodeCount'] for n in nodes]
# nodeWeight = [n['weight'] for n in nodes]

# edgeSources = [e['source'] for e in edges]
# edgeTargets = [e['target'] for e in edges]
# edgeLevels = [e['level'] for e in edges]
# edgeWeights = [e['weight'] for e in edges]

# virtualEdgeSources = [e['source'] for e in virtual_edges]
# virtualEdgeTargets = [e['target'] for e in virtual_edges]
# virtualEdgeWeights = [e['weight'] for e in virtual_edges]
# virtualEdgeHops = [e['hops'] for e in virtual_edges]


---

## Error Analysis

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)


def json_to_errors(fn, fn_node, shouldScale=False):
    with open(fn) as f:
        data = json.load(f)
    with open(fn_node) as f:
        nodes = json.load(f)

    label_to_id = {l:i for l,i in zip(data['node_label'], data['node_id'])}
    id_to_label = {v:k for k,v in label_to_id.items()}

    my_edges = [[id_to_label[i], id_to_label[j]] for i,j in zip(data['edge_source'], data['edge_target'])]
    edge_distance = {i:w for i,w in enumerate(data['edge_weight'])}
#     nodes_to_levels = {label: level for label, level in zip(data['node_label'], data['node_level'])}

    crd_x = {n['id']:n['x'] for n in nodes}
    crd_y = {n['id']:n['y'] for n in nodes}

#     label_to_xy = {l: (crd_x[i], crd_y[i]) for l,i in label_to_id.items()}
    xy = np.array([[crd_x[k], crd_y[k]] for k in crd_x])

    lines = [
        [(crd_x[label_to_id[l1]], crd_y[label_to_id[l1]]), 
        (crd_x[label_to_id[l2]], crd_y[label_to_id[l2]])] 
        for l1,l2 in my_edges
    ]

    edge_coords = np.array(lines)
    s, t = edge_coords[:, 0], edge_coords[:, 1]
    actual_length = np.linalg.norm(s - t, axis=1)
    ideal_length = np.array(list(edge_distance.values()))

    if shouldScale:
        num = np.sum(ideal_length**2/actual_length)
        den = np.sum(ideal_length)
        s = num / den
    else:
        s = 1
    relative_error = (actual_length*s - ideal_length) / ideal_length
    
    return lines, relative_error



def fn_coord_to_xy(fn_coord):
    with open(fn_coord) as f:
        xy = [l.split()[:2] for l in f]
        xy = [(float(x), float(y)) for x,y in xy]
    return xy



def fn_edge_to_lines(fn_edge, xy):
    
    with open(fn_edge) as f:
        f.readline()##skip first (comment) line
        f.readline()##skip second line
        reads = [l.split()[:3] for l in f]
        ideal_length = [float(r[2]) for r in reads]
        edges = [r[:2] for r in reads]
        edges = [(int(i), int(j)) for i,j in edges]
        lines = [(xy[i-1], xy[j-1]) for i,j in edges]
    return lines, ideal_length



def txt_to_errors(fn_coord, fn_edge, shouldScale=False):
    ## for large graphs
    
    xy = fn_coord_to_xy(fn_coord)
    lines, ideal_length = fn_edge_to_lines(fn_edge, xy)
        
    edge_coords = np.array(lines)
    s, t = edge_coords[:, 0], edge_coords[:, 1]
    actual_length = np.linalg.norm(s - t, axis=1)
    ideal_length = np.array(ideal_length)
    if shouldScale:
        num = np.sum(ideal_length**2/actual_length)
        den = np.sum(ideal_length)
        s = num / den
    else:
        s = 1
    relative_error = (actual_length*s - ideal_length) / ideal_length
    return lines, relative_error


    
def error_analysis(lines, relative_error, 
                   figsize=[8,15], height_ratios=[3,1], 
                   xlim=None, ylim=None):
    rows, cols = 2, 1
    gs = gridspec.GridSpec(rows, cols, width_ratios=[1,], height_ratios=height_ratios)
    gs.update(left=0.05, right=0.95, wspace=0.1, hspace=0.1)

    vmax = 1
    cm = plt.cm.get_cmap('coolwarm_r')

    # Plot histogram.
    plt.figure(figsize=figsize)

    ax = plt.subplot(gs[0,0])
    # ax.scatter(xy[:,0], xy[:,1], s=1, zorder=2)
    lc = mc.LineCollection(lines, 
                           colors=[cm((e+vmax) / (vmax*2)) for e in relative_error], 
                           linewidths=1)
    ax.add_collection(lc)
    ax.autoscale()
    ax.margins(0.1)
    plt.axis('equal')
    plt.xticks([])
    plt.yticks([])
    
    if xlim is not None:
        ax.set_xlim(xlim)
    if ylim is not None:
        ax.set_ylim(ylim)
        
    plt.subplot(gs[1,0])
    n, bins, patches = plt.hist(relative_error, color='green', bins=60)
    bin_centers = 0.5 * (bins[:-1] + bins[1:])

    # scale values to interval [0,1]
    col = (bin_centers + vmax ) / (vmax*2)

    for c, p in zip(col, patches):
        plt.setp(p, 'facecolor', cm(c))
    
    
    
    plt.xlabel('relative error')
    plt.ylabel('# edges')

    


In [ ]:
# dir_in = './data/json/topics-5000-low-degree/old/'
# fn = 'topics-8-sfdp'
# version = 16



#-------------------------------------------
# paper topics
# dir_in = './data/json/TopicsLayersData-0'
# fn = 'Graph_5000'
# version = 3

# paper lastfm
# dir_in = './data/json/lastfm/'
# fn = 'Graph_8'
# version = 2

#-------------------------------------------

##demo topics
# dir_in = './data/json/TopicsLayersData-0'
# fn = 'Graph_5000-radial'
# version = 7

##demo lastfm
# dir_in = './data/json/lastfm/'
# fn = 'Graph_8'
# version = 12

#-------------------------------------------
## demo, post-pacific-vis submission

##topics
# fn = './data/json/topics_refined/Graph_5000-min.json'
# fn_node = './data/json/topics_refined/Graph_5000-nodes-3.json'

# ##lastfm
# fn = './data/json/lastfm_refined/Graph_8_2587-min.json'
# fn_node = './data/json/lastfm_refined/Graph_8_2587-nodes-1.json'

#-------------------------------------------
## VIS 2021 submission
shouldScale = False

## CG

# fn = 'data/json/lastfm_linear/Graph_8-1615803307-min.json'
# fn_node = './data/json/lastfm_linear/Graph_8-1615803307-nodes-1.json'
# lines, relative_error = json_to_errors(fn, fn_node, shouldScale=shouldScale)

# fn = 'data/json/topics_faryad_8level_linear/Graph_5000-1615834916-min.json'
# fn_node = 'data/json/topics_faryad_8level_linear/Graph_5000-1615834916-nodes-3.json'
# lines, relative_error = json_to_errors(fn, fn_node, shouldScale=shouldScale)

# fn = 'data/json/tol_graphs_linear/Graph_4-1615872482-min.json'
# fn_node = 'data/json/tol_graphs_linear/Graph_4-1615872482-nodes-1.json'
# lines, relative_error = json_to_errors(fn, fn_node, shouldScale=shouldScale)

## 
def js_to_errors(fn, level_to_dist, shouldScale=False):
#     g = read_graph(graph_dir)
    data = read_js(fn)
    crd_x, crd_y = data['crd_x'], data['crd_y']
    label_to_id = data['label_to_id']
    my_edges = data['my_edges']
    
    nodes_to_levels = data['nodes_to_levels']
    edge_distance = {
        i:level_to_dist[
            max(
                nodes_to_levels[my_edges[i][0]], 
                nodes_to_levels[my_edges[i][1]]
            )]
        for i in range(len(my_edges))
    }
    xy = np.array([[crd_x[k], crd_y[k]] for k in crd_x])
    lines = [
        [(crd_x[str(label_to_id[l1])], crd_y[str(label_to_id[l1])]), 
        (crd_x[str(label_to_id[l2])], crd_y[str(label_to_id[l2])])] 
        for l1,l2 in my_edges
    ]

    edge_coords = np.array(lines)
    s, t = edge_coords[:, 0], edge_coords[:, 1]
    actual_length = np.linalg.norm(s - t, axis=1)
    ideal_length = np.array(list(edge_distance.values()))

    if shouldScale:
        num = np.sum(ideal_length**2/actual_length)
        den = np.sum(ideal_length)
        s = num / den
    else:
        s = 1
    relative_error = (actual_length*s - ideal_length) / ideal_length
    
    return lines, relative_error


def read_js(fn_in, center=True, scale=1):
    data_in = {}
    with open(fn_in) as f0:
        for line in f0:
            if line.startswith('//'):
                continue
            var_name, value = line.split(' = ')
            value = value.strip()
            try:
                value = json.loads(value)
            except json.JSONDecodeError as e:
                print(e)
                print(repr(value[e.colno-5:e.colno+20]))
                return e
            data_in[var_name] = value

    print(data_in.keys())
    return data_in

level_to_dist = {i:200+50*(8-i) for i in range(1,9)}
# fn = '../mlgd/map_generator/in/lastfm-DELG.json'
# lines, relative_error = js_to_errors(fn, level_to_dist, shouldScale=shouldScale)
# fn = '../mlgd/map_generator/in/topics-DELG.json'
# lines, relative_error = js_to_errors(fn, level_to_dist, shouldScale=shouldScale)
fn = '../mlgd/map_generator/in/tol-DELG.json'
lines, relative_error = js_to_errors(fn, level_to_dist, shouldScale=shouldScale)

## BT
# fn_coord = 'data/large/topics/Graph_28.top.txt.weighted.mtxBatchPrEL128PARAOUT0.txt'
# fn_edge = 'data/large/topics/Graph_28.top.txt.weighted.mtx'
# lines, relative_error = txt_to_errors(fn_coord, fn_edge, shouldScale=shouldScale)

In [ ]:
# !pip install pygraphviz

import pygraphviz as pgv

dot_fn = './data/external/in/lastfm/direct_lastfm_8.dot'
gv = pgv.AGraph(dot_fn, strict=False, directed=False)

G = nx.Graph(gv)
for i, node in tqdm(G.nodes(data=True)):
    node['pos'] = node['pos'].split(',')
    node['pos'] = [float(node['pos'][0]), float(node['pos'][1])]
    
pos = {i:node['pos'] for i,node in G.nodes(data=True)}
draw(G, pos, figsize=[4,4], s=1);

## grap attributes

In [ ]:
node_id = natsorted(pos.keys())
node_x = [pos[n][0] for n in node_id]
node_y = [pos[n][1] for n in node_id]
node_index = list(range(len(node_id)))
node_level = [1] * len(node_id)
node_label = [G.nodes[i]['label'] for i in G.nodes]

edges = G.edges
edge_source = [int(e[0]) for e in edges]
edge_target = [int(e[1]) for e in edges]




out_fn = 'data/external/lastfm-CIR.json'

## write output json

In [ ]:
out = dict(
    node_x=node_x,
    node_y=node_y,
    node_id=node_id,
    node_index=node_index,
    node_label=node_label,

    node_level=node_level, 

    edge_source=edge_source,
    edge_target=edge_target,
)
with open(out_fn, 'w') as f:
    json.dump(out, f)